## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15632417057264793089
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7129736807
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16864341856143264329
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('train : x=%s, y=%s' %(x_train.shape,y_train.shape))
print('test : x=%s, y=%s'  %(x_test.shape,y_test.shape))

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std

## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

train : x=(50000, 32, 32, 3), y=(50000, 1)
test : x=(10000, 32, 32, 3), y=(10000, 1)


In [9]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [ ]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape = (32,32,3),activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))


#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) 
classifier.add(BatchNormalization())
classifier.add(Dropout(0.5))

classifier.add(Dense(100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
classifier.add(BatchNormalization())

classifier.add(Dropout(0.3))

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)

from tensorflow.keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop
##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])

Epoch 1/100
500/500 [==============================] - 20s 41ms/step - loss: 2.1248 - accuracy: 0.3515 - val_loss: 1.6496 - val_accuracy: 0.5080
Epoch 2/100
500/500 [==============================] - 20s 39ms/step - loss: 1.7052 - accuracy: 0.4664 - val_loss: 1.5026 - val_accuracy: 0.5300
Epoch 3/100
500/500 [==============================] - 20s 39ms/step - loss: 1.5287 - accuracy: 0.5191 - val_loss: 1.3986 - val_accuracy: 0.5623
Epoch 4/100
500/500 [==============================] - 19s 39ms/step - loss: 1.4287 - accuracy: 0.5459 - val_loss: 1.2602 - val_accuracy: 0.5973
Epoch 5/100
500/500 [==============================] - 19s 38ms/step - loss: 1.3698 - accuracy: 0.5604 - val_loss: 1.2623 - val_accuracy: 0.5899
Epoch 6/100
500/500 [==============================] - 19s 38ms/step - loss: 1.3405 - accuracy: 0.5688 - val_loss: 1.1671 - val_accuracy: 0.6215
Epoch 7/100
500/500 [==============================] - 19s 39ms/step - loss: 1.3113 - accuracy: 0.5781 - val_loss: 1.2003 - val_ac

Epoch 29/100
500/500 [==============================] - 19s 39ms/step - loss: 1.1655 - accuracy: 0.6386 - val_loss: 1.0911 - val_accuracy: 0.6640
Epoch 30/100
500/500 [==============================] - 19s 39ms/step - loss: 1.1642 - accuracy: 0.6352 - val_loss: 1.0238 - val_accuracy: 0.6813
Epoch 31/100
500/500 [==============================] - 19s 39ms/step - loss: 1.1615 - accuracy: 0.6367 - val_loss: 1.0735 - val_accuracy: 0.6654
Epoch 32/100
500/500 [==============================] - 19s 38ms/step - loss: 1.1593 - accuracy: 0.6412 - val_loss: 1.0379 - val_accuracy: 0.6799
Epoch 33/100
500/500 [==============================] - 19s 39ms/step - loss: 1.1671 - accuracy: 0.6355 - val_loss: 1.1135 - val_accuracy: 0.6566
Epoch 34/100
500/500 [==============================] - 19s 39ms/step - loss: 1.1549 - accuracy: 0.6414 - val_loss: 1.0724 - val_accuracy: 0.6649
Epoch 35/100
500/500 [==============================] - 19s 38ms/step - loss: 1.1551 - accuracy: 0.6392 - val_loss: 0.9733 -

Epoch 57/100
265/500 [==============>...............] - ETA: 8s - loss: 1.1342 - accuracy: 0.6517

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)